In [2]:
import dask
import dask.array as da

In [3]:
PATH = 'myfirstpath.hdf5'
DATASET_KEY = "/data"

In [11]:
def get_array():
    with h5py.File(PATH, 'r') as f:
        dataset = f[DATASET_KEY]
        return da.from_array(dataset, chunks=(100, 100, 100))

In [12]:
def create_array():
    random_array = da.random.random(size=(500, 500, 500))
    da.to_hdf5(PATH, DATASET_KEY, random_array, chunks=None, compression=None)

In [21]:
create_array()

# case 1 

In [15]:
import h5py
arr = get_array()
arr = arr[slice(0, 400), slice(0, 200), slice(0, 100)]
arr = arr + 21
graph = arr.__dask_graph__()
graph.dependencies

{'add-5c59f4501b025d614eb3b19e1dad07f1': {'getitem-4757cb6c8bb3c3058f1e84d993bfd6d1'},
 'getitem-4757cb6c8bb3c3058f1e84d993bfd6d1': {'array-e646486de5a1cca449ac013817900a77'},
 'array-e646486de5a1cca449ac013817900a77': set()}

# case 2 

In [23]:
import h5py
PATH2 = 'mysecondpath.hdf5'
arr = get_array()
arr_list = [arr[0:100, 0:100, :100], arr[100:200, 0:100, :100], arr[200:300, 0:100, :100]]

datasets = list()
with h5py.File(PATH2, 'a') as f:
    for i, a in enumerate(arr_list):
        key = '/data' + str(i)
        datasets.append(f.create_dataset(key, shape=a.shape))

arr = da.store(arr_list, datasets, compute=False)
graph = arr.__dask_graph__()
graph.dependencies

{140359199321936: set(),
 140359199317280: set(),
 140359199330944: set(),
 140359199995216: set(),
 140359199319776: set(),
 140359199739248: set()}